# 🚀 **Aula 02: Parte 02 - Coletando e trabalhando com dados**

## Nesta aula iremos estar aprendendo como coletar dados de alguns websites e como trabalhar com esses dados, e para isso, estaremos utilizando a biblioteca **Beautiful Soup** para realizar a **raspagem de dados**, a biblioteca **requests** para realizar as requisições e por último, utilizaremos a biblioteca **pandas** para trabalhar com os dados.

---

# **1. A Biblioteca requests** ⚡

## A primeira coisa que precisamos fazer para realizar o web scrap é baixar a página. Podemos baixar as páginas utilizando a biblioteca requests do Python. <br>
## A biblioteca requests fará uma solicitação GET ao servidor, que fará o download dos conteúdos HTML da página solicitada para nós. <br> 
## Existem vários tipos de solicitação diferentes que podemos realizar utilizando a biblioteca requests – GET é apenas um deles.

# **2. A Biblioteca BeautifulSoup** ⚡

<img src="images/beautifulSoup.jpg" alt="Drawing" style="width: 600px;"/>

## Depois de realizarmos uma requisicao bem sucedida da página que desejamos usando o requests, podemos utilizar a biblioteca BeautifulSoup para analisar o html da página e extrair o conteúdo das tags que desejamos.

# **3. A biblioteca Pandas** ⚡

<img src="images/pandas.png" alt="Drawing" style="width: 400px;"/>

## **pandas** é uma biblioteca de software criada para a linguagem Python para manipulação e análise de dados. <br>
## Iremos utilizar-lá para manipular os dados coletados durante a raspagem de dados.

# **4. Instalando bibliotecas** ⚡

## Antes de importar as bibliotecas precisamos realizar a instalacao das mesmas. <br>
## Para isso, iremos utilizar o gerenciador de pacotes do Python ***pip***

In [5]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install seaborn

You should consider upgrading via the 'C:\Users\Antonio Costa - Dev\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Antonio Costa - Dev\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Antonio Costa - Dev\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Antonio Costa - Dev\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


## **4.1 Importando pacotes**

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import seaborn as sns

# **5 Criando nosso primeiro programa** ⚡

## 5.1 Coletando dados do portal de notícias da Unipinhal

### Antes de comecar a coleta, vamos verificar o site para coletar algumas informacoes

In [ ]:
def scrapping_unipinhal():
    url = 'https://www.sou.unipinhal.edu.br/noticias'
    website = requests.get(url)
    if website.status_code != 200:
        return None
    # conteudo da requisicao
    conteudo_website = website.content
    # analise do resultado obtido
    website_analisado = BeautifulSoup(conteudo_website, 'html.parser')
    # Div contendo todos valores desejados
    informacoes = website_analisado.find_all("div", attrs= {"class":"_3tJ3x _1e-gz post-list-item-wrapper blog-post-homepage-description-font blog-post-homepage-description-color blog-post-homepage-description-fill _3RzkT"})
    # criacao de lisdta vazia para ser preenchida com o tempo
    informacoes_completas = []
    for informacao in informacoes:
        titulo = informacao.select('h2 div')[0]
        descricao = informacao.find('div', attrs = {"class": "_81XUh"})
        data = informacao.select('li span')[0]
        informacoes_completas.append([titulo.text, descricao.text, data.text])
    return informacoes_completas

In [ ]:
lista_de_informacoes = scrapping_unipinhal()

In [ ]:
df = pd.DataFrame(lista_de_informacoes, columns = ['Titulo', 'Descricao', 'Data'])
df

In [ ]:
df.to_excel('NoticiasUnipinhal.xlsx', index = False)

# **6. Coletando e trabalhando com dados do website imbd.com** ⚡

## **6.1 Buscando filmes: Coletando dados de 200 filmes pelo genero escolhido**

##

In [4]:
def busca_filmes(genero):
    # url dos filmes a serem buscados
    url = f'https://www.imdb.com/find?s=tt&q={genero}&ref_=nv_sr_sm'
    website = requests.get(url)
    # condicao: se o codigo for diferente de 200 faca com que a funcao nao retorne nada (parar a funcao)
    if website.status_code != 200:
        return None
    #seguimento padrao (coletando conteudo do website)
    cont_web = website.content
    #seguimento padrao (analisando o conteudo do website)
    website_analisado = BeautifulSoup(cont_web, 'html.parser')
    #titulo, link do filme
    informacoes = website_analisado.findAll("td", attrs = {"class": "result_text"})
    informacoes_filmes = []
    for informacao in informacoes:
        #coletando o a da div (a div possui apenas uma tag a, logo, buscando por ela nesse ionetrvalo conseguimos extrair o texto dela.)
        titulo = informacao.find('a').text
        # coletando link
        link_filme = informacao.find('a')['href']
        #Como estamos fazendo o append com mais de um item, 'e necessario criar uma lista dentro do append, por isso os colchetes
        informacoes_filmes.append([titulo, link_filme])
        print(titulo, link_filme)

filme = input("Digite um genero de filme")
busca_filmes(filme)

Ação Mutante /title/tt0106215/
Ação no Quênia /title/tt0130350/
Ação Direta /title/tt0368688/
Ação e Reação /title/tt1261862/
Ação Imediata /title/tt1182609/
The Umbrella Academy /title/tt1312171/
Acapulco /title/tt13567344/
Boku no Hero Academia /title/tt5626028/
Amizade de Férias /title/tt3626476/
Gattaca, uma Experiência Genética /title/tt0119177/
Macao /title/tt0044863/
Férias Frustradas /title/tt1524930/
Boku no Hero Academia: World Heroes Mission /title/tt13544716/
Hotel Transilvânia 3 - Férias Monstruosas /title/tt5220122/
Loucademia de Polícia /title/tt0087928/
Ação/Reação /title/tt1180168/
Dupla Ação /title/tt1123534/
Câmera... Ação! /title/tt6393794/
Ação & Games /title/tt8822882/
Ação Policial /title/tt7779038/
Férias Frustradas /title/tt0085995/
Férias Frustradas de Natal /title/tt0097958/
Temos Vagas /title/tt0452702/
As Espiãs de Churchill /title/tt7698468/
Academia de Vampiros: O Beijo das Sombras /title/tt1686821/
Curacao /title/tt0106633/
Dinheiro Fácil: A Série /title

## **6.2 Coletar ano de producao e score de todos os 200 filmes ja coletados**

### Para isso, vamos utilizar um filme exemplo para coletar os dados

In [ ]:
link_exemplo = f"https://www.imdb.com/title/tt0130350/?ref_=fn_tt_tt_2"
request_exemplo = requests.get(link_exemplo)
analise = BeautifulSoup(request_exemplo.content, 'html.parser')

## **Coletando o ano**

In [ ]:
ano = analise.find("span", attrs = {"class": "TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex"})
print(ano.text)

### Funcao que retorna o ano de todos os filmes, de acordo com o parametro *analise_site*

In [ ]:
def get_ano(analise_site):
    ano =  analise_site.find("span", attrs = {"class": "TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex"})
    try:
        return (int(ano.text))
    except Exception:
        return None

## **Coletando score**

In [ ]:
score = analise.find("span", attrs = {"class": "AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV"})
print(score.text)

### Funcao que retorna o score de todos os filmes, de acordo com o parametro *analise_site*

In [ ]:
def get_score(analise_site):
    score = analise_site.find("span", attrs = {"class": "AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV"})
    if score is None:
        return None
    return float(score.text)

## **Reunindo as funcoes:**

In [3]:
def get_ano(analise_site):
    ano =  analise_site.find("span", attrs = {"class": "TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex"})
    try:
        return (int(ano.text))
    except Exception:
        return None

def get_score(analise_site):
    score = analise_site.find("span", attrs = {"class": "AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV"})
    if score is None:
        return None
    return float(score.text)

def buscador_filmes(genero, numero_filmes):
    url = f'https://www.imdb.com/find?s=tt&q={genero}&ref_=nv_sr_sm'
    website = requests.get(url)
    if website.status_code != 200:
        return None
    cont_web = website.content
    website_analisado = BeautifulSoup(cont_web, 'html.parser')
    #titulo, link do filme
    informacoes_filmes = website_analisado.findAll("td", attrs = {"class": "result_text"})
    titulos_filmes = []
    links_completos = []

    # Primeiro for: Coletar titulos e links dos filmes
    for informacao_filme, i in zip(informacoes_filmes, range(numero_filmes)):
        titulo = informacao_filme.find('a').text
        link_padrao = "https://www.imdb.com"
        link_filme = informacao_filme.find('a')['href']
        link_completo = link_padrao + link_filme
        titulos_filmes.append(titulo)
        links_completos.append(link_completo)

    lista_filmes = []
    # Segundo for: com os links coletados fazer uma iteracao em cada um e coletar as informacoes das funcoes get_score() e get_ano())
    for titulo, link_completo in zip(titulos_filmes, links_completos):
        dic = {'titulo': titulo, 'link': link_completo}
        req = requests.get(link_completo)
        analise = BeautifulSoup(req.content, 'html.parser')
        req.close()

        dic['score'] = get_score(analise)
        dic['ano'] = get_ano(analise)
        lista_filmes.append(dic)

    print(lista_filmes)
genero_filme = input("Digite um genero de filme ")
qtt_filmes = int(input("Quantos filmes voce deseja coletar? "))

buscador_filmes(genero_filme, qtt_filmes)

[{'titulo': 'Rogue, o Assassino', 'link': 'https://www.imdb.com/title/tt0499556/', 'score': 6.2, 'ano': 2007}, {'titulo': 'Guerra', 'link': 'https://www.imdb.com/title/tt7430722/', 'score': 6.5, 'ano': 2019}, {'titulo': 'Guerra', 'link': 'https://www.imdb.com/title/tt3830162/', 'score': 7.1, 'ano': 2015}, {'titulo': 'Guerra dos Tronos', 'link': 'https://www.imdb.com/title/tt0944947/', 'score': 9.2, 'ano': None}, {'titulo': 'Guerra nas Estrelas', 'link': 'https://www.imdb.com/title/tt0076759/', 'score': 8.6, 'ano': 1977}, {'titulo': 'O Império Contra-Ataca', 'link': 'https://www.imdb.com/title/tt0080684/', 'score': 8.7, 'ano': 1980}, {'titulo': 'Guerra Mundial Z', 'link': 'https://www.imdb.com/title/tt0816711/', 'score': 7.0, 'ano': 2013}, {'titulo': 'Guerra dos Mundos', 'link': 'https://www.imdb.com/title/tt0407304/', 'score': 6.5, 'ano': 2005}, {'titulo': 'Homeland', 'link': 'https://www.imdb.com/title/tt1796960/', 'score': 8.3, 'ano': None}, {'titulo': 'Irmãos de Guerra', 'link': 'ht